

### Generación de Modelos Predictivos con LOGISTIC REGRESSION

Este tipo de modelos requiere una revisión previa de colienalidad.  Tambien permiten obtener información del peso de las variables en el modelo.

In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import VarianceThreshold

# Cargar tu DataFrame, reemplaza 'tu_dataframe.csv' con el nombre de tu archivo CSV
dfbase = pd.read_pickle('checkpoints/DATOS_LIMPIOS.pkl')

Cargamos el archivo ya preparado de DATOS_LIMPIOS y separamos las variables independientes en **X** y la dependiente en **y**

In [3]:
df = dfbase.copy()

In [4]:
df = df[df['LABEL']!=-1]

In [5]:
X = df.drop('LABEL', axis=1)
y = df['LABEL']

Obtenemos la matriz de correlación de las variables independientes con la idea de eliminar aquellas variables muy correlacionadas entre si.  Dado que la matriz es simétrica, se deja solo la mitad inferior

In [6]:
# calculamos correlación de variables independientes
corr_matrix = X.corr()

import numpy as np
mitad_inferior = np.tril(corr_matrix, k=-1)
mitad_inferior = pd.DataFrame(mitad_inferior, columns=corr_matrix.columns, index=corr_matrix.index)

Se realiza un estudio por umbral, para determinar cuantas variables serían eliminadas y cuantas quedarían.   Entre más alto el umbral, las variables independientes están menos correlacionadas entre si

In [7]:
from sklearn.metrics import roc_auc_score, roc_curve
from sklearn.metrics import accuracy_score

lista_umbral = [0.9, 0.8, 0.7, 0.6, 0.5, 0.4, 0.3, 0.2, 0.1]

for umbral in lista_umbral:
    j = 0
    variables_eliminar = set()
    for c in mitad_inferior.columns:
        i = 0
        for v in mitad_inferior[c]:
            if v > umbral:
                variables_eliminar.add(mitad_inferior.index[i])           
            i = i + 1
        j = j + 1
    print('umbral:', 
          umbral, len(mitad_inferior.columns), 
          len(variables_eliminar),
          len(mitad_inferior.columns) - len(variables_eliminar))
    
    Xnew = X.drop(list(variables_eliminar), axis=1)

    # Separar los datos en conjuntos de entrenamiento y prueba
    X_train, X_test, y_train, y_test = train_test_split(Xnew, y, test_size=0.2, random_state=42)

    # BALANCAMOS X_train y y_train
    #!pip install imbalanced-learn
    from imblearn.over_sampling import SMOTE
    smote = SMOTE(sampling_strategy='auto')
    X_train, y_train = smote.fit_resample(X_train, y_train)


    print(y_test.value_counts())

    # Aplicar StandardScaler para estandarizar las características (opcional pero recomendado)
    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.transform(X_test)

    # Entrenar el modelo de regresión logística
    model = LogisticRegression(max_iter=3000)
    cv_scores = cross_val_score(model, X_train, y_train, cv=5)  # Utiliza 5 divisiones para validación cruzada

    # Imprimir los puntajes de validación cruzada
    print("Puntajes de validación cruzada:", cv_scores)
    print("Precisión media: {:.2f}".format(cv_scores.mean()))

    model.fit(X_train, y_train)

    y_pred_prob = model.predict_proba(X_test)[:, 1]
    y_pred = model.predict(X_test)
    roc_auc = roc_auc_score(y_test, y_pred_prob)
    accuracy_test = accuracy_score(y_test, y_pred)

    print('roc-auc:', roc_auc, 'accuracy:', accuracy_test)



            


umbral: 0.9 1953 490 1463
LABEL
0    970
1    353
Name: count, dtype: int64
Puntajes de validación cruzada: [0.79090909 0.83181818 0.92597403 0.92597403 0.92792208]
Precisión media: 0.88
roc-auc: 0.8851347799421746 accuracy: 0.8458049886621315
umbral: 0.8 1953 584 1369
LABEL
0    970
1    353
Name: count, dtype: int64
Puntajes de validación cruzada: [0.79350649 0.83246753 0.92402597 0.92727273 0.91753247]
Precisión media: 0.88
roc-auc: 0.8860576501854502 accuracy: 0.8382464096749811
umbral: 0.7 1953 703 1250
LABEL
0    970
1    353
Name: count, dtype: int64
Puntajes de validación cruzada: [0.76623377 0.80974026 0.91168831 0.92792208 0.92337662]
Precisión media: 0.87
roc-auc: 0.8771677229052889 accuracy: 0.8321995464852607
umbral: 0.6 1953 799 1154
LABEL
0    970
1    353
Name: count, dtype: int64
Puntajes de validación cruzada: [0.75974026 0.81493506 0.91298701 0.91298701 0.90779221]
Precisión media: 0.86
roc-auc: 0.8728541806606115 accuracy: 0.81859410430839
umbral: 0.5 1953 915 1038
